**Общая информация**

**Срок сдачи:** 16 марта 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [327]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from scipy.spatial.distance import cdist
from sklearn import neighbors
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups
from sklearn.base import ClassifierMixin
from sklearn.neighbors import KDTree

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [328]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors ,algorithm='brute'):
        self.algorithm=algorithm
        self.n_neighbors = n_neighbors
    
    def fit(self, X, y):
        if(X.shape[0] != y.shape[0]): #количество меток = количество наблюдений
            pass
        self.y = np.asarray(y)
        self.labels_num = np.unique(y).shape[0]
        self.X = [[]]
        if self.algorithm == 'kd_tree':
            self.kdtree = KDTree(X,metric = 'euclidean')
        elif self.algorithm == 'brute':
            self.X = X
    
    def predict(self, X):
        if self.algorithm == 'kd_tree':
            array_of_neighbors = self.kdtree.query(X,k = self.n_neighbors,return_distance = False)
            def winner(a):
                return self.y[np.argmax(np.bincount(a))]
            return np.apply_along_axis(winner,1,array_of_neighbors)
        elif self.algorithm == 'brute':
            distances_array = cdist(X,self.X)
            def brute_search(a):
                return np.argmax(np.bincount(self.y[np.argpartition(a,self.n_neighbors)[:self.n_neighbors]]))
            return np.apply_along_axis(brute_search,1,distances_array)

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [329]:
iris = datasets.load_iris()

In [330]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [331]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [332]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [333]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [334]:
%time clf.fit(X_train, y_train)

CPU times: user 3.26 ms, sys: 0 ns, total: 3.26 ms
Wall time: 2.42 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [335]:
%time my_clf.fit(X_train, y_train)

CPU times: user 156 µs, sys: 8 µs, total: 164 µs
Wall time: 105 µs


In [336]:
%time clf.predict(X_test)

CPU times: user 3.63 ms, sys: 0 ns, total: 3.63 ms
Wall time: 2.55 ms


array([1, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 2, 0])

In [337]:
%time my_clf.predict(X_test)

CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 1.43 ms


array([1, 2, 1, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 2, 0])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [338]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [339]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [340]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 2.63 ms, total: 2.63 ms
Wall time: 4.92 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [341]:
%time my_clf.fit(X_train, y_train)

CPU times: user 455 µs, sys: 23 µs, total: 478 µs
Wall time: 508 µs


In [342]:
%time clf.predict(X_test)

CPU times: user 9.45 ms, sys: 4.35 ms, total: 13.8 ms
Wall time: 9.4 ms


array([1, 0, 0, 0, 2, 1, 2, 0, 0, 1, 1, 2, 2, 1, 2])

In [343]:
%time my_clf.predict(X_test)

CPU times: user 1.23 ms, sys: 64 µs, total: 1.3 ms
Wall time: 942 µs


array([1, 0, 0, 0, 2, 1, 2, 0, 0, 1, 1, 2, 2, 1, 2])

In [344]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

In [345]:
accuracy_score(y_test, my_clf_pred)

1.0

In [346]:
accuracy_score(y_test, sklearn_pred )

1.0

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [280]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [285]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [322]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)


*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [326]:
from sklearn.metrics import pairwise_distances

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [ ]:
class Sparse_KNN(BaseEstimator):
    
    def __init__(self,n_neighbors,algorithm = 'brute',metric = 'euclidean'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.metric = metric
        
    def 
#не смог...        

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.